In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

2025-09-06 17:17:53.970511: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-06 17:17:54.022280: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/dpxk/miniconda3/envs/dlenv/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
2025-09-06 17:17:55.244611: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
# Load the trained model, encoders and scalers 
model = load_model('model.keras')

with open('gender_le.pkl', 'rb') as file: 
    gender_le = pickle.load(file)

with open('geography_ohe.pkl', 'rb') as file: 
    geography_ohe = pickle.load(file)

with open('scaler.pkl', 'rb') as file: 
    scaler = pickle.load(file)

E0000 00:00:1757159276.281931    7098 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1757159276.286699    7098 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Sample input data
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000,
}

In [4]:
# apply ohe to geography
geo_encoded = geography_ohe.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geography_ohe.get_feature_names_out(['Geography']))
geo_encoded_df

/home/dpxk/miniconda3/envs/dlenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
# Convert input_data into dataframe
df = pd.DataFrame([input_data])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
# Encode categorical variables
df['Gender'] = gender_le.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
# Concat ohe df and the input df
df = pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
# Scale the input data
scaled_df = scaler.transform(df)
scaled_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [9]:
# Predict churn
prediction = model.predict(scaled_df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


array([[0.01793174]], dtype=float32)

In [10]:
prediction_probability = prediction[0][0]
prediction_probability

np.float32(0.017931739)

In [11]:
if prediction_probability > 0.5: 
    print("Customer is likely to churn.")
else: 
    print("Customer is not likely to churn.")

Customer is not likely to churn.
